We're interested in a mirror that is reflective in the mid-IR (reflective
band centered 1550 - 2200 cm-1 depending on exact system, with reflectivity
band width of ~200 cm-1 or more) and as transmissive as possible in the
upper visible (roughly from 320-700 nm, but we could settle for a smaller
range).

There are quite a few materials at our disposal either in-house or through a
vendor.  I could list them all but I suppose we could start with the most
common.
Ag
Si3N4
SiO2
TiO2
Al2O3
Ta2O5
ZrO2

*from Michael*


I also think it could be desirable if the near-UV to visible is transparent, the mid-IR from ~4000 cm^-1 to 500 cm^-1 is completely reflective,

*from Blake* 
While it is true that the width of the reflective region in the mid-IR would "ideally" be somewhat narrow (this would remove unintended couplings), I'd say that is a secondary concern.  Broadly speaking, we're not too concerned about what happens between ~2200cm-1 and 700nm.

# Can a DBR work?
If we center the reflection band in the mid-IR, can we get lucky enough to have high transmissivity in 
the visible band?  We can start with a guess structure based on this concept and
try to optimize the thicknesses from there.

We will base the guess thicknesses on using glass ($n \approx 1.5$) and aluminum oxide ($n \approx 1.85$) using
$$ nd=\frac{\lambda}{4} \rightarrow d = \frac{\lambda}{4n}$$

with $\lambda = 5 \mu m$.

In [ ]:
import wptherml
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# guess thickness for glass
d1 = 5000e-9 / (4 * 1.5)
# guess thickness for Al2O3
d2 = 5000e-9 / (4 * 1.85)

print(F"Trial thickness of glass layer is {d1:.3e} m")
print(F"Trial thickness of alumina layer is {d2:.3e} m")

We will set up a structure with 13 periods of alternating SiO2 and Al2O3 (26 layers total).

The spectra will be computed in the wavelength range $300 \: nm - 6000 \: nm$.

We will set the desired window of high reflectivity to be between $2000 \: cm^{-1} - 2400 \: cm^{-1}$
(equivalently, $4166 \: nm - 5000 \: nm$.)

We will compute two different figures of merit that we will call the transmission efficiency ($\eta_T$)
and the reflection efficiency ($\eta_R$).

We will define the transmission efficiency as 

$$ \eta_T = \frac{\int_{\lambda_1}^{\lambda_2} \Pi_T(\lambda) T(\lambda) d\lambda }{\int_{\lambda_1}^{\lambda_2} \Pi_T(\lambda) d\lambda} $$
where $T(\lambda)$ is the transmission spectrum of the stack and $\Pi_T(\lambda)$ is the desired 
transmission envelope (defined to be 1 in the range specified by the `transmissive_window_nm` keyword and 0 elsewhere; note the user specifies this in units of nanometers.  The default is 350 - 700 nm).

The reflection efficiency can be defined as
$$ \eta_R = \frac{\int_{\lambda_1}^{\lambda_2} \Pi_R(\lambda) R(\lambda) d\lambda }{\int_{\lambda_1}^{\lambda_2} R(\lambda) d\lambda} $$
where $R(\lambda)$ is the reflection spectrum of the stack and $\Pi_R(\lambda)$ is the desired 
reflection envelope (defined to be 1 in the range specified by the `reflective_window_wn` keyword and 0 elsewhere,
note the user speciies this in units of inverse centimeters.  The default is 2000 - 2400 $cm^{-1}$).

The wavelength range of the spectra and integral are set by the `wavelength_list` keywords.

The structure composition is set by the `Material_List` keyword.

The structure geometry is set by the `Thickness_List` keyword.

The block below will create an instance of a DBR that we will call `test`.  

We will call the method `test.compute_selective_mirror_fom()` to compute $\eta_T$ and $\eta_R$, as well as a 
composite figure of merit called `test.selective_mirror_fom` defined as $f = \frac{1}{2} \left( \eta_T + \eta_R \right)$




In [ ]:
test_args = {
    "wavelength_list": [300e-9, 6000e-9, 1000],
    "Material_List": ["Air","SiO2", "Al2O3", "SiO2", "Al2O3","SiO2", "Al2O3", "SiO2", "Al2O3","SiO2", "Al2O3", "SiO2", "Al2O3","SiO2", "Al2O3","SiO2", "Al2O3", "SiO2", "Al2O3","SiO2", "Al2O3", "SiO2", "Al2O3" ,"SiO2", "Al2O3", "SiO2", "Al2O3" , "Air"],
    "Thickness_List": [0,d1, d2, d1, d2,d1, d2, d1, d2, d1, d2, d1, d2, d1, d2,d1, d2, d1, d2, d1, d2, d1, d2, d1, d2, d1, d2, 0],
    "reflective_window_wn" : [2000, 2400],
    "transmissive_window_nm" : [350, 700],
 }

sf = wptherml.SpectrumFactory()

# create an instance of the DBR called test
test = sf.spectrum_factory('Tmm', test_args)

# compute the foms
test.compute_selective_mirror_fom()

print(F'Reflection Efficiency is {100 * test.reflection_efficiency:.2f} %')
print(F'Transmission Efficiency is {100 * test.transmission_efficiency:.2f} %')
print(F'Composite FOM is {100 * test.selective_mirror_fom:.2f} %')


We can plot the spectra on a wavenumber and wavelength axis, respectively, to visualize the behavior:

In [ ]:

plt.plot(test.wavenumber_array * 1e-2, test.transmissive_envelope, label='T window')
plt.plot(test.wavenumber_array * 1e-2, test.reflective_envelope, label='R window')
plt.plot(test.wavenumber_array * 1e-2, test.transmissivity_array, label='T')
plt.plot(test.wavenumber_array * 1e-2, test.reflectivity_array, label='R')
plt.plot(test.wavenumber_array * 1e-2, test.emissivity_array, label='A')
plt.legend()

In [ ]:
plt.plot(test.wavelength_array, test.transmissive_envelope, label='T window')
plt.plot(test.wavelength_array, test.reflective_envelope, label='R window')
plt.plot(test.wavelength_array, test.transmissivity_array, label='T')
plt.plot(test.wavelength_array, test.reflectivity_array, label='R')
plt.plot(test.wavelength_array, test.emissivity_array, label='A')
plt.legend()



We can now attempt to optimize the layer thicknesses to improve $\eta_T$, $\eta_R$, or some weighted average
of the two.  We are going to use the built-in BFGS optimizer in scipy that will attempt to find the minimimum 
of some objective `f(x)` given the variable(s) `x`.  

We will create two helper functions: `update_multilayer(x)` will take
an array of thicknesses (in nanometers) for the stack and recompute the figures of merit at this geometry, and
`superFunc(x)` will be a function that passes an updated `x` from the BFGS routine to the `update_multilayer(x)` 
function and return the `f(x)` value to the BFGS routine.  For now, we will not rely on gradients ($f'(x)$),
we will let the BFGS estimate then numerically, but if we add this functionality, it can be called upon and
passed back to BFGS by `superFunc`.

**Note 1** Because BFGS will want to minimize, we will actually have `superFunc` return the negative of our objectives so that BFGS will minimize the negative of our objective, which will yield a maximum of our objetive.

**Note 2** We don't want to accidentally go to ridiculous value of the thickness, so we can set bounds on the
BFGS (which is actually provided by a variant of the BFGS routine called L-BFGS-B).  We will set the lower bound 
to be 1 nm and the upper bound to be 1000 nm. You can experiment with this!



In [ ]:
# library containing L-BFGS-B
from scipy.optimize import minimize

# this is a library for global optimization - can try later! from scipy.optimize import basinhopping
import time


# start the spectrum driver
sf = wptherml.SpectrumFactory()
# create an instance using the TMM with the structure defined as above
test = sf.spectrum_factory('Tmm', test_args)

def update_multilayer(x):
    """ function to update the thicknesses of each layer given an
        array of thicknesses stored in x"""
    test.thickness_array[1:test.number_of_layers-1] = x * 1e-9
    test.compute_spectrum()
    test.compute_selective_mirror_fom()
    
    # We have three choices for what to define our FOM as 
    # choice 1: \eta_R
    fom = test.reflection_efficiency
    
    #choice 2: \eta_T
    #fom = test.transmission_efficiency
    
    # choice 3: average of \eta_R + \eta_T
    #fom = test.selective_mirror_fom
    
    # return negative of fom
    return -1 * fom


def SuperFunc(x0):
    en = update_multilayer(x0)
    print(x0, en)
    return en


# the bounds for L-BFGS-B updates
# minimum layer thickness is 1 nm
bfgs_xmin = np.ones(test.number_of_layers-2)

# maximum layer thickness is 1000 nm
bfgs_xmax = 1000*np.ones(test.number_of_layers-2)

# rewrite the bounds in the way required by L-BFGS-B
bfgs_bounds = [(low, high) for low, high in zip(bfgs_xmin, bfgs_xmax)]

# define d1 and d2 in nanometers based on the trial values from first cell
d1 = 835
d2 = 675

# define trial x array
xs = np.array([d1, d2, d1, d2,d1, d2, d1, d2, d1, d2, d1, d2, d1, d2,d1, d2, d1, d2, d1, d2, d1, d2, d1, d2, d1, d2])
print("xs is ")
print(xs)
fom = update_multilayer(xs)
print("initial FOM is ", fom)

### run l-bfgs-b algorithm, jac=False means L-BFGS-B will estimate gradients numerically
ret = minimize(SuperFunc, xs, method="L-BFGS-B", jac=False, bounds=bfgs_bounds)

### print optimal solution and its efficiency!
print("Optimal Structure")
print(ret.x)
print("Optimal FOM")
print(update_multilayer(ret.x))

In [ ]:
print(test.thickness_array)

In [ ]:

plt.plot(test.wavenumber_array * 1e-2, test.transmissive_envelope, label='T window')
plt.plot(test.wavenumber_array * 1e-2, test.reflective_envelope, label='R window')
plt.plot(test.wavenumber_array * 1e-2, test.transmissivity_array, label='T')
plt.plot(test.wavenumber_array * 1e-2, test.reflectivity_array, label='R')
plt.plot(test.wavenumber_array * 1e-2, test.emissivity_array, label='A')
plt.legend()

In [ ]:
plt.plot(test.wavelength_array, test.transmissive_envelope, label='T window')
plt.plot(test.wavelength_array, test.reflective_envelope, label='R window')
plt.plot(test.wavelength_array, test.transmissivity_array, label='T')
plt.plot(test.wavelength_array, test.reflectivity_array, label='R')
plt.plot(test.wavelength_array, test.emissivity_array, label='A')
plt.legend()

In [ ]:
bfgs_xmin = np.ones(test.number_of_layers-2)
# maximum layer thickness is 400 nm
bfgs_xmax = 800*np.ones(test.number_of_layers-2)

# rewrite the bounds in the way required by L-BFGS-B
bfgs_bounds = [(low, high) for low, high in zip(bfgs_xmin, bfgs_xmax)]

bfgs_bounds[test.number_of_layers-3] = (1.0, 5.0)
print(bfgs_bounds)

In [ ]:
### prints efficiency and time
def print_fun(x, f, accepted):
    c_time = time.time()
    print(f,",",c_time)

### called by the basin hopping algorithm to initiate new
### local optimizations
def my_take_step(x):
    xnew = np.copy(x)
    dim = len(xnew)
    for i in range(0,dim):
        rn = 50.*np.abs(np.random.randn())
        xnew[i] = rn
    return xnew

test.number_of_layers-2
### bounds on basin hopping solutions
class MyBounds(object):
      ### note xmax and xmin need to have as many elements as there are thicknesses that are varied
    def __init__(self, xmax=800.0*np.ones(test.number_of_layers-2), xmin=1.0*np.ones(test.number_of_layers-2)):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

# the bounds for L-BFGS-B updates!
bfgs_xmin = 1.0*np.ones(test.number_of_layers-2)
bfgs_xmax = 800.0*np.ones(test.number_of_layers-2)

# rewrite the bounds in the way required by L-BFGS-B
bfgs_bounds = [(low, high) for low, high in zip(bfgs_xmin, bfgs_xmax)]

### arguments for basin hopping algorithm
minimizer_kwargs = {"method": "L-BFGS-B", "jac": False, "bounds": bfgs_bounds}
mybounds = MyBounds()

### initial guess for AR layer thicknesses!
xs = np.array([209.4868565,  726.08020804, 800.,           2.1529940])

### call basin hopping!
ret = basinhopping(SuperFunc, xs, minimizer_kwargs=minimizer_kwargs, niter=100, take_step=my_take_step, callback=print_fun, accept_test=mybounds)

### print optimimal result!
print(ret.x)
print(update_multilayer(ret.x))

In [ ]:
plt.plot(test.wavelength_array, test.transmissive_envelope, label='T window')
plt.plot(test.wavelength_array, test.reflective_envelope, label='R window')
plt.plot(test.wavelength_array, test.transmissivity_array, label='T')
plt.plot(test.wavelength_array, test.reflectivity_array, label='R')
plt.plot(test.wavelength_array, test.emissivity_array, label='A')
plt.legend()